In [1]:
import sys
import os
import pandas as pd
sys.path = [os.path.abspath('..')] + sys.path

In [3]:
from utils import settings, data_loader

In [8]:
data = data_loader.get_raw_data()

In [9]:
data.keys()

dict_keys(['student_assessment', 'student_info', 'student_vle', 'courses', 'vle', 'student_registration', 'assessments'])

In [10]:
data['student_assessment'].head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [55]:
train, test = train_test_split(data['student_assessment'])

In [12]:
data['student_info']['region'].value_counts()

Scotland                3446
East Anglian Region     3340
London Region           3216
South Region            3092
North Western Region    2906
West Midlands Region    2582
South West Region       2436
East Midlands Region    2365
South East Region       2111
Wales                   2086
Yorkshire Region        2006
North Region            1823
Ireland                 1184
Name: region, dtype: int64

In [60]:
test.shape

(52174, 5)